This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'rWjv3zpy3j_dY-Ax52sd'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# Query paramaters
start_date = '2017-01-01'
end_date = '2017-01-01'

#Building URL for api query
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?' + 'start_date=' + start_date + '&end_date=' + end_date + '&api_key=' + API_KEY

#Query server
res = requests.get(url)


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

request_result = res.json()
print(list(request_result.keys()))

#after seeing one nested key, that layer can be peeled off. 
dataset = request_result['dataset']
print(list(dataset.keys()))

# We can see that the data is mostly under the nested location dataset > data, which is a list of lists containing data for each day
# Titles for the lists can be found in column_names
# Other information from the query also avialable

['dataset']
['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
"""Task #1"""
# Query paramaters. Dates YYYY-MM-DD
start_date = '2017-01-01'
end_date = '2017-12-31'

#Building URL for api query
url='https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?' + 'start_date=' + start_date + '&end_date=' + end_date + '&api_key=' + API_KEY

#Query server
res = requests.get(url)

In [6]:
"""Task 2: Import into python dictonary. dataset is python dictonary and stock_data is pandas dataframe"""    
#Unpack Data
request_result = res.json()
print(list(request_result.keys()))
dataset = request_result['dataset']
print(list(dataset.keys()))

stock_data = json_normalize(dataset, 'data')
stock_data.columns = dataset['column_names']
stock_data['Date'] = pd.to_datetime(list(stock_data['Date']))
stock_data = stock_data.reindex(index=stock_data.index[::-1])
stock_data = stock_data.set_index('Date')

['dataset']
['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id']


In [7]:
"""Task 3: Calculate highest and lowest opening prices in this period"""
max_open = stock_data['Open'].max()
min_open = stock_data['Open'].min()

In [8]:
"""Task 4: Calculate largest single day change"""
stock_data['Delta'] = stock_data['High']-stock_data['Low']
largest_change = stock_data['Delta'].max()

In [9]:
"""Task 5: Calculate change over any 2 days on closing price"""
#Allowing for the shifted day by offsetting data by 1
close_end = stock_data['2017-01-03':]
close_begin = stock_data['2017':'2017-12-28']

#Removing date index so subtraction doesnt subtract by date
close_end = close_end.reset_index()
close_begin = close_begin.reset_index()    
    
#Calculate maximum change in closing price. Absolute value used to account for potnetial for maximum change being negative. 
day_difference = close_end['Close']-close_begin['Close']  
max_difference = day_difference.abs().max()

In [10]:
"""Task 6: Average daily trading volume"""
avg_daily_volume = stock_data['Traded Volume'].mean()

In [11]:
"""Task 7: Median Trading Volume"""
mean_daily_volume = stock_data['Traded Volume'].median()